# Wrangling OpenStreetMap data
*Updated: 11 Dec 2015*

---

## Project details:

The goal of this project was to get my hands dirty with OpenStreetMap data. OSM is an open source platform, so the quality of data for any given area depends hugely on the community of users that is inputing and editing it. I dug into the NYC data ([this area](https://www.openstreetmap.org/relation/175905#map=10/40.6983/-73.9792)) because this is one of the most developed parts of the OSM dataset.

To simplify and speed things up, I downloaded the greater NYC map dataset from [MapZen Metro Extracts](https://mapzen.com/data/metro-extracts/). This extract actually included nearby areas of the states of NY, NJ, and CT, not just the New York City area outlined in the OSM link above.

## Processing and auditing the data

In [3]:
# import relevant modules

import xml.etree.ElementTree as ET 
from lxml import html
import requests
from collections import defaultdict
import re
import pprint
import codecs
import json

### Decision to use sample of NYC data set

The MapZen metro extract for NYC actually included parts of neighboring cities and regions, including from nearby states. Given the large size and density of this region, the extract was huge in size - 2.17 GB. My machine wasn't powerful enough to process a data set this large repeatedly, so I decided to take a sample of the data set and use that for auditing, cleaning, and analysis instead. As the code below shows, I took read in every 30th top level element. This provided a sample data set of 73.4 MB, which was more manageable for my machine.

In [94]:
# download sample of osm file

OSM_FILE = "new-york_new-york.osm"
SAMPLE_FILE = "sample.osm"


def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every 30th top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % 30 == 0:
            output.write(ET.tostring(element, encoding='utf-8'))
    output.write('</osm>')

In [4]:
# function to scrape New York City zipcodes from government website

def get_zipcodes(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)

    zip_list = tree.xpath('//td[@headers="header3"]/text()')
    zipcodes = set()
    for i in range(0, len(zip_list)):
        temp = zip_list[i].split(',')
        for elem in temp:
            elem = elem.lstrip()
            zipcodes.add(elem)
    return zipcodes

In [96]:
# process sample data, output potential problems with the data

# small sample of OSM data set
osmfile = "sample.osm"

# set up a few regex tools to simplify regex application later on
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
alph = re.compile(r'[a-zA-Z]', re.IGNORECASE)

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
EXPECTED_STREET = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", 
                   "Square", "Lane", "Road", "Trail", "Parkway", "Commons", "Plaza", 
                   "Terrace", "Way"]
EXPECTED_CITY = ['New York']


# url of NYC zipcodes from government website
NYC_ZIPCODES = get_zipcodes('https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm')


# check to see if element is street name
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")                


# add street type to dictionary if unexpected
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in EXPECTED_STREET:
            street_types[street_type].add(street_name)

            
# check to see if element is zipcode
def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")                


# add postcode to dictionary if unexpected
def audit_postcode(postcodes, not_nyc_postcodes, postcode):
    if (problemchars.match(postcode)) or (alph.match(postcode)) or (len(postcode) != 5):
        postcodes.add(postcode)
    elif postcode not in NYC_ZIPCODES:
        if postcode not in not_nyc_postcodes.keys():
            not_nyc_postcodes[postcode] = 1
        else:
            not_nyc_postcodes[postcode] = not_nyc_postcodes[postcode] + 1


# check to see if element is city name        
def is_city(elem):
    return (elem.attrib['k'] == "addr:city")                


# add city name to dictionary if unexpected
def audit_city(cities, city):
    if city not in EXPECTED_CITY:
        if city not in cities.keys():
            cities[city] = 1
        else:
            cities[city] = cities[city] + 1


# takes element and shapes it and its tags into a dictionary
def shape_element(element, street_types, postcodes, not_nyc_postcodes, cities):
    node = {}
    addr = {}
    created = {}
    pos = [0,0]
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        for a in element.attrib:
            if a in CREATED:
                created[a] = element.get(a)
            elif a == 'lat':
                pos[0] = float(element.get(a))
            elif a == 'lon':
                pos[1] = float(element.get(a))
            else:
                node[a] = element.get(a)
        for t in element.iter("tag"):
            if problemchars.match(t.get('k')):
                pass
            elif t.get('k').startswith('addr:'):                
                if is_street_name(t):
                    audit_street_type(street_types, t.attrib['v'])
                elif is_postcode(t):
                    audit_postcode(postcodes, not_nyc_postcodes, t.attrib['v'])
                elif is_city(t):
                    audit_city(cities, t.attrib['v'])
                a = t.get('k').split(':')
                if len(a) > 2:
                    pass
                else:
                    addr[a[1]] = t.get('v')
            else:
                node[t.get('k')] = t.get('v')
        if len(addr) > 0:
            node['address'] = addr
        if len(created) > 0:
            node['created'] = created
        if pos[0] > 0:
            node['pos'] = pos
                    
        return node
        return street_types
        return postcodes
        return not_nyc_postcodes
    else:
        return None


# main function to launch processing and auditing of sample data
def process_map(file_in, pretty = False):
    data = []
    street_types = defaultdict(set)
    postcodes = set()
    not_nyc_postcodes = defaultdict(set)
    cities = defaultdict(set)
    for _, element in ET.iterparse(file_in):
        el = shape_element(element, street_types, postcodes, not_nyc_postcodes, cities)
        if el:
            data.append(el)
    return [data, street_types, postcodes, not_nyc_postcodes, cities]              

data, street_types, postcodes, not_nyc_postcodes, cities = process_map(osmfile, False)

## Problems encountered in the map data:
I encountered inconsistencies and problems in the quality of the data in three areas:
- street types;
- zipcodes; and
- city names.

#### Problems with street types:
My audit of unexpected street types revealed several things. 

First, I discovered a number of street types that were perfectly acceptable. This included street types like 'circle', 'crescent', and 'loop'. It also included the lettered avenues (e.g., Avenue A, Avenue B), which are acceptable names in New York City.

Second, I discovered a number of problematic cases that would need to be cleaned up. A good example is the presence of 'Avenue' (in my expected list), 'AVENUE', 'Ave', and 'avenue', all of which should conform to a single spelling and capitalization. Another example is the presence of fully capitalized spellings (e.g., COLLEGE AVENUE, GIBBONS CIRCLE).

In [97]:
# print unexpected street types found in sample data

pprint.pprint(dict(street_types))

{'10': set(['Nwe Jersey 10']),
 '35': set(['NJ Route 35']),
 'A': set(['Avenue A']),
 'AVENUE': set(['COLLEGE AVENUE', 'NICHOL AVENUE']),
 'Alley': set(['Grace Court Alley']),
 'Americas': set(['Avenue Of The Americas']),
 'Ave': set(['Willow Ave']),
 'B': set(['Avenue B']),
 'Bayside': set(['Bayside']),
 'Blvd': set(['Northern Blvd', 'Queens Blvd']),
 'Bowery': set(['Bowery']),
 'Broadway': set(['Broadway', 'East Broadway', 'West Broadway']),
 'C': set(['Avenue C']),
 'CIRCLE': set(['GIBBONS CIRCLE']),
 'Center': set(['World Financial Center']),
 'Circle': set(['64th Circle',
                'Alex Circle',
                'Beech Court Circle',
                'Beekman Circle',
                'Country Pointe Circle',
                'Covington Circle',
                'Cromwell Circle',
                'Dawson Circle',
                'Field Point Circle',
                'Flagship Circle',
                'Fort Hill Circle',
                'Hugh J Grant Circle',
                'Mar

#### Problems with zipcodes:
My audit of zipcodes revealed three problematic areas: 

First, while editors usually enter a 5-digit zipcode, some enter the full 9-digit zip code that provides even greater granularity. While the 9-digit version is correct as far as postal codes goes, it introduces heterogeneity in format that will present problems for analysis.

Second, my audit revelaed several cases of incorrectly inputed zipcodes, such as 'NY' and 'NY 10002'. The former is simply an incorrect data type, while the latter includes the state name where it should not.

Third, given that the data extract downloaded from MapZen covers a region larger than just New York City, there is obviously a lot of data not related to the city that i want to investigate. We know this from the beginning, but we don't know how big of an issue it is. Looking at city names is indicative (see below for more on this), but not ideal because there are many small towns and neighborhoods from inside and outside New York City that I don't recognize. By contrast, zipcodes provide an easier and more objective solution to this problem. I wrote a script to scrape the list of zipcodes for New York City from a government website. I then checked zipcodes in the OSM data set against this government list in order to identify which zipcodes were from outside the city and how frequently they showed up in the data. 

In [98]:
# print unexpected zipcodes found in sample data 

pprint.pprint(postcodes)

set(['08854-5659',
     '08854-8001',
     '08854-8006',
     '08854-8007',
     '08854-8009',
     '08854-8010',
     '08901-1175',
     '08901-1414',
     '08901-1573',
     '08901-2867',
     '08901-2882',
     '08901-8505',
     '08901-8529',
     '08901-8531',
     '08901-8540',
     '08901-8542',
     '08901-8556',
     '08901-8557',
     '74',
     'NY 10002',
     'NY 10533'])


In [99]:
# counts of zipcodes not included in official list of NYC zipcodes scraped from government website

pprint.pprint(dict(not_nyc_postcodes))

{'06807': 1,
 '06830': 1,
 '06831': 1,
 '06855': 1,
 '06902': 1,
 '06905': 1,
 '07024': 2,
 '07030': 1,
 '07033': 1,
 '07040': 1,
 '07042': 1,
 '07043': 5,
 '07090': 2,
 '07095': 3,
 '07105': 1,
 '07302': 5,
 '07401': 1,
 '07444': 1,
 '07450': 2,
 '07457': 1,
 '07504': 1,
 '07508': 1,
 '07604': 1,
 '07621': 1,
 '07624': 1,
 '07735': 2,
 '07901': 1,
 '07927': 1,
 '07932': 1,
 '07981': 1,
 '08816': 6,
 '08820': 1,
 '08830': 1,
 '08840': 1,
 '08854': 3,
 '08873': 1,
 '08901': 3,
 '10069': 1,
 '10281': 1,
 '10282': 2,
 '10522': 1,
 '10583': 1,
 '10591': 1,
 '10604': 1,
 '11001': 44,
 '11003': 1,
 '11030': 1,
 '11040': 22,
 '11096': 1,
 '11109': 1,
 '11249': 71,
 '11566': 1,
 '11581': 1,
 '11717': 1,
 '11754': 1,
 '11758': 1,
 '11787': 1,
 '11801': 1,
 '11803': 1}


#### Problems with city names:
My audit of city names revealed several problems: 

First, as noted above, the New York City metro extract downloaded from MapZen actually covers a wider region than just NYC. This means that a number of legitimate cities that are not 'New York City' show up in the data.

Second, we see in this list several entries for the boroughs of New York City (e.g., Bronx, Brooklyn). These are not cities, but instead parts of New York City.

Third, a number of city names include the state abbreviation (e.g., 'Brooklyn, NY', 'Fresh Meadows, NY'). The state abbreviations are erroneous elements of these entries.

In [100]:
# print city names

pprint.pprint(dict(cities))

{'Allendale': 1,
 'Astoria': 3,
 'Bergenfield': 1,
 'Bronx': 6,
 'Brooklyn': 51,
 'Brooklyn, NY': 1,
 'Cedar Knolls': 1,
 'Closter': 1,
 'Corona': 2,
 'Cos Cob': 1,
 'Denville': 1,
 'Dobbs Ferry': 1,
 'Dumont': 1,
 'East Brunswick': 4,
 'East Brunswick Township': 1,
 'East Norwalk': 1,
 'Edgewater': 1,
 'Florham Park': 1,
 'Forest Hills': 1,
 'Fort Lee': 2,
 'Fresh Meadows NY': 1,
 'Glendale, NY': 1,
 'Greenwich': 2,
 'Harrison': 1,
 'Hasbrouck Heights': 1,
 'Hoboken': 15,
 'Irvington': 7,
 'Iselin': 1,
 'Jamaica': 1,
 'Jersey City': 4,
 'Kenilworth': 1,
 'Keyport': 2,
 'Kings Park': 1,
 'Long Island City': 2,
 'Madison': 1,
 'Manhasset': 1,
 'Maplewood': 1,
 'Merrick, New York': 1,
 'Montclair': 6,
 'New Brunswick': 15,
 'New Hyde Park': 1,
 'New York City': 8,
 'New York NY': 1,
 'Newark': 1,
 'North Haledon': 1,
 'North Hills': 1,
 'Paramus': 1,
 'Paterson': 1,
 'Piscataway': 9,
 'Queens': 1,
 'Ridgewood': 2,
 'Riverdale': 1,
 'Smithtown': 1,
 'Somerset': 1,
 'Stamford': 2,
 'Summit

## Cleaning the data and storing it in JSON

Given the problems discovered in the data, I made the following cleaning decisions:
- expand my list of expected street types to include those unexpected entries that are perfectly acceptable;
- clean street types that are out of order, as noted above;
- retain only 5-digit zipcodes. Where zipcodes include the full 9 digits, strip the last 4 digits;
- clean up zipcodes and city names so no state abbreviations are present;
- retain non-NYC zipcodes and city names. While these areas are not in NYC, they remain fine from a map perspective, so I chose to leave them alone;
- change city names that refer to boroughs to instead refer to the name of the city, 'New York'.

In [5]:
# process and clean the full data set, save to JSON format

# replace with full OSM data file
osmfile = "sample.osm"

problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
PROBLEM_CITY = ['Bronx', 
                'Brooklyn', 
                'Queens', 
                'Manhattan', 
                'Staten Island', 
                'New York City',
                'NYC',
                'NY']
STATES = ['NY', 'CT', 'NJ', 'PA']
street_type_mapping = {'St.': 'Street', 
                       'avenue': 'Avenue', 
                       'LANE': 'Lane', 
                       'AVENUE': 'Avenue', 
                       'Blvd': 'Boulevard',
                       'EAST': 'East',
                       'ROAD': 'Road',
                       'St': 'Street',
                       'Ave': 'Avenue',
                       'Pky': 'Parkway',
                       'CIRCLE': 'Circle',
                       'Ct': 'Court'}


# checks to see if element is street name
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")                


# clean street type
def audit_street_type(street_name):
    if street_name.split()[-1] in street_type_mapping.keys():
        street = street_name.split()
        street[-1] = street_type_mapping[street_name.split()[-1]]
        s = ""
        for i in range(0,len(street)):
            s = s + " " + street[i]
    else:
        s = street_name
    return s

          
# checks to see if element is zipcode
def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")                


# clean zipcode
def audit_postcode(postcode):
    zipcode = ''.join(e for e in postcode if (e.isalnum()))
    zipcode = re.sub('[^0-9]+', '', zipcode)
    zipcode = zipcode[0:5]
    
        
# checks to see if element is city name        
def is_city(elem):
    return (elem.attrib['k'] == "addr:city")                


# clean city name
def audit_city(city):   
    for elem in STATES:
        city = city.split(',')[0].replace(elem, '').strip()
    city = city.title()
    if city in PROBLEM_CITY:
        city = 'New York'
    return city
        

# takes element and shapes it and its tags into a dictionary
def shape_element(element):
    node = {}
    addr = {}
    created = {}
    pos = [0,0]
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        for a in element.attrib:
            if a in CREATED:
                created[a] = element.get(a)
            elif a == 'lat':
                pos[0] = float(element.get(a))
            elif a == 'lon':
                pos[1] = float(element.get(a))
            else:    
                node[a] = element.get(a)
        for t in element.iter("tag"):
            if '.' in t.get('k'):
                pass
            elif problemchars.match(t.get('k')):
                pass
            elif t.get('k').startswith('addr:'):                
                if is_street_name(t):
                    addr['street'] = audit_street_type(t.attrib['v'])
                elif is_postcode(t):
                    audit_postcode(t.attrib['v'])
                elif is_city(t):
                    audit_city(t.attrib['v'])
                a = t.get('k').split(':')
                if len(a) > 2:
                    pass
                else:
                    addr[a[1]] = t.get('v')
            else:
                node[t.get('k')] = t.get('v')
        if len(addr) > 0:
            node['address'] = addr
        if len(created) > 0:
            node['created'] = created
        if pos[0] > 0:
            node['pos'] = pos
                    
        return node
    else:
        return None


# main function to launch processing and auditing of sample data
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data              

data = process_map(osmfile, False)

## Import clean JSON file to MongoDB, run queries against it

In [6]:
# import JSON data to MongoDB collection

from pymongo import MongoClient

def get_db():    
    client = MongoClient(host='localhost', port=27017)
    db = client.nyc
    return db


def insert_data(data, db):
    db.nyc.insert_many(data)
    return db
  

db = get_db()
insert_data(data, db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'nyc')

## Data Overview

2.17 GB, kernel died.

File sizes:
- osm file: 73.4 MB
- osm.json file: 74.1 MB

In [7]:
# Number of documents
print "Total number of documents: ", db.nyc.count()

Total number of documents:  570015


In [8]:
# Number of nodes
print "Total number of nodes: ", db.nyc.find({'type': 'node'}).count()

Total number of nodes:  494671


In [9]:
# Number of ways
print "Total number of ways: ", db.nyc.find({'type': 'way'}).count()

Total number of ways:  75314


In [10]:
# Count unique contributors
print "Total unique contributors: ", len(db.nyc.distinct('created.user'))

Total unique contributors:  1394


In [20]:
# Top 5 contributors

result = db.nyc.aggregate([{'$group': {'_id': '$created.user',
                                       'count': {'$sum': 1}}},
                           {'$sort': {'count': -1}},
                           {'$limit': 5}
                          ])

print "Top 5 Contributors:"
for record in result:
    print "{:,}".format(record['count']), record['_id']

Top 5 Contributors:
261,293 Rub21_nycbuildings
63,146 woodpeck_fixbot
49,940 ingalls_nycbuildings
14,492 ediyes_nycbuildings
12,556 lxbarth_nycbuildings


In [30]:
# Count of establishments, grouped by amenity type

result = db.nyc.aggregate([{'$match': {'amenity': {'$exists': 1}}},
                           {'$group': {'_id': '$amenity', 
                                       'count':{'$sum': 1}}},
                           {'$sort': {'count': -1}},
                           {'$limit': 10}])

print "Top 10 Amenities:"
for record in result:
    print record['count'], record['_id']

Top 10 Amenities:
348 place_of_worship
346 school
282 parking
265 bicycle_parking
141 restaurant
46 hospital
42 bank
40 fast_food
37 fire_station
36 post_office


In [29]:
# Top 10 types of fast food establishments by cuisine type

print "Total amenities labelled 'fast food':", db.nyc.find({'amenity': 'fast_food'}).count(), '\n'

result = db.nyc.aggregate([{'$match': {'amenity': 'fast_food'}},
                           {'$group': {'_id': '$cuisine',
                                      'count': {'$sum': 1}}},
                           {'$sort': {'count': -1}},
                           {'$limit': 10}])

print "Top 10 types of fast food establishments by cuisine type:"
for record in result:
    print record['count'], record['_id']

Total amenities labelled 'fast food': 40 

Top 10 types of fast food establishments by cuisine type:
19 None
7 sandwich
5 burger
4 pizza
3 mexican
1 trini
1 chicken


## Ideas on how to use and improve the dataset

Student understands the process involved in proposed changes.
Student gives thoughtful discussion about the benefits as well as some anticipated problems in implementing the improvement.

The OSM data set could be used for a number of interesting projects. For example:
- The data could be used to identify which neighborhoods of a city have a particularly high or low density of supermarkets. Finding neighborhoods with a low density of supermarkets would help us find 'food deserts', where residents have poor access to fresh and diverse foods. Such an analysis would need to control for population density so that non-residential (e.g., industrial storage) areas, which naturally lack food amenities, are not mis-identified as 'food deserts'. Doing this kind of analysis would require having geo-spatial boundaries for neighborhoods.
- The data could be used in an application that helps users identify services (e.g., schools, hospitals, banks, post offices) by neighborhood. As with the previous example, using the data in this way would require a list of geo-spatial coordinates for neighborhood boundaries. As an alternative, the data could be used to show services within a certain geographical range of the user, providing the user shares their location coordinates.

The OSM data set could be improved by adding a much stronger economic and social overlay. For example:
- In terms of amenities, the most frequent category referred to in the data set is place of worship, followed closely by school (see list above). This list clearly lacks some of the most commonly found establishments in a city, such as stores and other private businesses. Compared to a service such as Google Maps, where a user can easily find many types of stores and services, the OSM data set is not yet useful for many of these use cases. One way to address this would be to scrape data on location type and postal address from a website like Yelp or Yellow Pages, and to programmatically input this data into the OSM data set. This would require significant work to scrape and transform the data, but the upside would be an OSM data set that would be vastly more useful for many use cases.
- In some cases where contributors have inputed data on businesses, the data is clearly incomprehensive. For example, certain contributors have used a 'fast food' classification for amenities, which helps us identify fast food establishments in a city. Yet, in the New York City region, only 40 such entries exist in the data, and among these almost half have no cuisine type listed. The data that is available is therefore often lacking key characteristics to make it useful to users. As with the above example, one way to address this would be to scrape data on location type and postal address from a website like Yelp or Yellow Pages, and to programmatically input this data into the OSM data set.